# ANOVA - Analysis of Variance
We will continue our discussion on linear model. Today, we will be learning ANOVA, a generalized form of comparing mean across multiple groups. 
Agenda today:
- Compare t-tests and ANOVA
- Learn to calculate ANOVA & details 
- Implement ANOVA in python
    - using statsmodel
    - from scratch
- Compare linear regression with ANOVA

## Part I. T tests or ANOVA?
Suppose we want to compare whether multiple groups differ in some type of measures. For example, we have collected mood data grouped by four types of weather - sunny, raining, overcast, or cloudy, and we want to find out whether there is a difference in mood across different weather. What tests would you use?

A natural reaction would be to conduct multiple t-tests. However, that comes with many drawbacks. First, you would need $\frac{n(n-1)}{2}$ t tests, which come out to 6 tests. Having more tests meaning having higher chance of making type I error. In this case, our original probability of making type I error grew from 5% to 5% x 6 = 30%! By conduct 6 tests and comparing their mean to each other, we are running a huge risk of making false positives. How then, can we combat this? -- ANOVA!

Instead of looking at each individual difference, ANOVA examines the ratio of variance between groups, and variance within groups, and find out whether the ratio is big enough to be statistically significant. 

#### T Test statistics 
$t = \frac{x\bar - \mu}{\frac{s}{\sqrt n}}$

#### ANOVA - the F test
$F = \frac{MS_bet}{MS_within}$

We can also say that t test is a special case of ANOVA in that we are comparing the means of only two groups.

## Part II. Calculating ANOVA 
In this section, we will learn how to calculate ANOVA without using any pacakges. All we need to calculate is:

- $SS_b$ = $n\sum(\bar X - \bar X_i)^2 $

- $SS_w$ = $\sum (n_i - 1) s_i ^ 2$

- $SS_t$ = $\sum (X_ij - \bar X)^2$

- $MS_b$ = $\frac{SS_b}{DF_b}$

- $MS_w$ = $\frac{SS_w}{DF_w}$

- $F$ = $\frac{MS_b}{MS_w}$

Degrees of Freedom for ANOVA:
- DFbetween = k - 1
- DFwithin = N - k
- DFtotal = N - 1

Notations:
- k is the number of groups
- N is the total number of observations
- n is the number of observations in each group

###
Like regression and t-test, we can also perform hypothesis testing with ANOVA. What would be the null hypothesis in this case? 
- $H0 = \mu_1 = \mu_1 = \mu_3 = \mu_4$
- $H_a$ H0 is not true

In [ ]:
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
data = pd.read_csv('bikeshare_day.csv')
data.head()
# cnt is the outcome we are trying to predict

In [ ]:
#Create a boxplot
# 1 is spring, 2 is summer, 3 is fall, and 4 is winter
data.boxplot('cnt', by='season', figsize=(6,6))

### 2.2 Calculate ANOVA using statsmodel

In [ ]:
# we need to conduct a little bit feature engineering to encode 
data['season_cat'] = data.season.apply(lambda x: 'spring' if x == 1 else ('summer' if x == 2 else ('fall' if x ==3 else 'winter')))

In [ ]:
data.boxplot('cnt', by = 'season_cat')

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm
# why can we use ols in this case?
anova_season = ols('cnt~season_cat',data=data).fit()
anova_season.summary()

In [ ]:
# examine the anova table 
anova_table = sm.stats.anova_lm(anova_season, type=2)
print(anova_table)

## Part III. Comparing Linear Regression to ANOVA
Why can we use ordinary least squares with ANOVA? ANOVA, in fact, can be viewed as a linear regression problem with categorical variable, parametrized as with j levels:
$y_i = \beta_0 + \beta_1 X_1 + ... + \beta_j-1 X_j-1 + \epsilon$

Therefore, the above model can also be interpreted as special case of linear regression with only categorical predictor of season can . be parametrized as:
$Y = 5644.30 - 3040.17X_1 - 651.97X_2 - 916.14X_3$

The intercept, again, represents fall
We can then interpret it as:
- When the season is fall, Y has a mean value of 5644.30

- When the season is spring, $Y = 5644.30 - 3040.17 *1 - 651.97*0 - 916.14*0 = 2604.13$

- When the season is summer, $Y = 5644.30 - 3040.17 *0 - 651.97*1 - 916.14*0 = 4992.33$

- When the season is winter, $Y = 5644.30 - 3040.17 *0 - 651.97*0 - 916.14*1 = 5644.30$

In [ ]:
data[data.season ==3].cnt.mean()

In [ ]:
data[data.season == 1].cnt.mean()

In [ ]:
data[data.season == 2].cnt.mean()

In [ ]:
data[data.season == 3].cnt.mean()

Working with linear regression with categorical _and_ continuous predictors - how to interpret it?

#### Next steps
Just because we have rejected the null hypothesis, it doesn't mean we have conclusively showed which group is significantly different from which - remember, the alternative hypothesis is "the null is not true". 

We need to conduct post hoc tests for multiple comparison to find out which groups are different, the most prominent post hoc tests are:
- LSD (Least significant difference)
    - $t\sqrt \frac{MSE}{n^2}$
- Tukey's HSD 
    - $q\sqrt \frac{MSE}{n^2}$
    
After calculating a value for LSD or HSD, we compare each pair wise mean difference with the LSD or HSD difference. If the pairwise mean difference exceeds the LSD/HSD, then they are significantly different.